In [13]:
# Import necessary packages
import cv2
import numpy as np
import os
from os.path import isfile, join
import tensorflow as tf
import os
import time
from matplotlib import pyplot as plt
from skimage import color
from skimage import io
from keras.preprocessing.image import save_img
import imageio
import shutil
from PIL import Image

In [14]:
# Function to split a video in frames
def video_to_frames(input_video, output_frames):
    vidcap = cv2.VideoCapture(input_video)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read()
        if hasFrames:
            cv2.imwrite(output_frames+"/image"+str(count)+".jpg", image)     
        return hasFrames
    sec = 0
    frameRate = 1/25.0
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

In [15]:
# Function to create the required folders
def create_folder(folder):
    if os.path.exists(folder):
        shutil.rmtree(folder)
        os.mkdir(folder)
    else:
        os.mkdir(folder)
    print(folder, 'folder created.')

In [16]:
# Defining the path to folders
train_video_path = 'val_video.mp4'
original_frames_path = 'output/'
generated_path = 'generated/'

In [17]:
# Create the necessary folders
for folder in [original_frames_path, generated_path]:
    create_folder(folder)

output/ folder created.
generated/ folder created.


In [18]:
# Split the video in frames and store them in the output folder
video_to_frames(train_video_path, original_frames_path)

In [19]:
# Defining parameters
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [20]:
# Function to normalize an 256x256 image
def normalize(real_image):
  real_image = (real_image / 127.5) - 1
  return real_image

In [21]:
# Function to load the filenames of files in a folder
def load_labels (folder):
    labels = os.listdir(folder)
    labels.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    return labels

In [22]:
# Function to load the filenames of files in a folder with the folder name appended
def load_labels_folder(labels, folder):
    new_labels=[]
    for label in labels:
        label=folder+label
        new_labels.append(label)
    return new_labels

In [23]:
# Function to load, crop, resize and normalize an image
def load_image(image_file):
    real_image = tf.io.read_file(image_file)
    real_image = tf.image.decode_jpeg(real_image, channels=1)

    real_image = tf.image.resize_with_crop_or_pad(real_image,256, 256)

    real_image = tf.cast(real_image, tf.float32)

    real_image = normalize(real_image)

    return real_image

In [24]:
# Store the labels
labels = load_labels(original_frames_path)
labels_folder = load_labels_folder(labels, original_frames_path)

In [25]:
# Load the labels in a TF tensor
labels_tensor = tf.constant(labels_folder)
# Load the images, create batches and prefetch
dataset = tf.data.Dataset.from_tensor_slices(labels_tensor)
dataset = dataset.map(load_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(1)

In [26]:
# Function to generate colored images and save them
def generate_images(model, test_input, label):
  prediction = model(test_input, training=True)
  save_img(generated_path+label, tf.squeeze(prediction)*0.5 +0.5)

In [27]:
# Load the trained model
model = tf.keras.models.load_model('model_60_trained.h5')

In [28]:
# Review the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128, 128, 64) 1024        input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64, 64, 128)  131584      sequential[0][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 32, 32, 256)  525312      sequential_1[0][0]               
______________________________________________________________________________________________

In [29]:
# Iterate over the dataset and generate colorized images
iterator = iter(dataset)
for i in range(len(labels)):
    input = iterator.get_next()
    label= labels[i]
    generate_images(model, input, label)

In [30]:
# Function to join frames creating a video
def join_frames(pathIn, pathOut):
    fps = 23.98
    frame_array = []
    files = load_labels(pathIn)
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)   
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [31]:
# Create the colorized video
generated_video = 'colorized_video.mp4'
join_frames(generated_path, generated_video)

In [32]:
# Function to create a gif of the desired lenght
def make_gif(pathIn, start, end):
    import imageio
    images = []
    files = load_labels(pathIn)
    for filename in files[start:end]:
        images.append(imageio.imread(pathIn+filename))
    imageio.mimsave('movie.gif', images)

In [33]:
make_gif(generated_path, 3000, 4000)